# 23_네이버 쇼핑 상품 목록 수집

네이버 쇼핑에서 특정 키워드로 검색된 내용을 엑셀로 수집하기 

## #01. 필요한 모듈 참조

In [1]:
import requests
import urllib      
from bs4 import BeautifulSoup     
from pandas import DataFrame

## #02. 수집 준비 

### 1) 접속을 수행하기 위한 session 객체 생성

> 웹 페이지에서 데이터를 수집할 때 가장 처음에 위치해야 하는 코드 입니다.

In [2]:
# 접속 세션 만들기 
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"
session = requests.Session()
session.headers.update({'User-agent':user_agent, 'referer':None})

### 2) 접근할 페이지 주소(네이버 쇼핑 검색 결과 주소 패턴)

#### 필요한 변수들 정의 

In [3]:
keyword ="카메라"
max_page = 5 

#### 기본 URL

브라우저에 표시되는 검색 결과 URL에서 `?`전 까지만 지정

In [4]:
base_url = "https://search.shopping.naver.com/search/all.nhn"

####  파라미터 구조

2페이지부터 3~4 페이지 정도 직접 이동하여 주소를 확인한 후 패턴을 찾는다.

> origQuery=검색어&pagingIndex=2&pagingSize=40&viewType=list&sort=rel&frm=NVSHPAG&query=검색어

In [5]:
base_param = {"origQuery":keyword,"pagingIndex":1, "pagingSize":40, "viewType":"list", "sort":"rel","frm" :"NVSHPAG", "query":keyword}

## #03. 반복 수행으로 데이터 수집하기 

개별 상품 정보들을 딕셔너리 형태로 수집한 다음 미리 준비한 빈 리스트에 추가한다.

In [16]:
# 수집 결과를 누적할 빈 리스트 
data_list = []

# 원하는 페이지 수 만큼 반복
for page in range(1, max_page+1):
  
    # 1) 접속할 주소 준비하기 
    #  파라미터에서 접근할 페이지 번호 설정
    base_param['pagingIndex'] = page
    
    # URLEncoding
    query = urllib.parse.urlencode(base_param)
    
    # 접속할 주소 확정
    content_url = base_url + "?" + query

    # 출력으로 5줄 url이 나오는지 확인하기 
    # print(content_url)
    
    # 2) 페이지에 접속하여 상품정보 박스 가져오기 
    # 세션을 통해서 주소에 접속
    r = session.get(content_url)
    
    if r.status_code != 200:
        print("%d 에러가 발생했습니다." % r.status-conde)
        continue
        
    # 인코딩 설정        
    r.encoding="utf-8"      
    
    # 웹 페이지의 소스코드 HTML 분석 객체로 생성
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # 상품 정보 박스 추출
    # 클래스 속성값은 앞에 점(.)을 붙여서 접근 ->  동일한 구조를 갖는 영역이 여러 개라는 의미
    info_list = soup.select(".info")
    # print("%d 페이지의 상품 수: %d" % (page, len(info_list)) )
    
    # 3) 현제 페이지에서 추출된 상품 정보의 수 만큼 반복
    for info in info_list:
        #  하나의 삳품 정보를 보관할 디셔너리 
        item_dict = {}
        
        # 3-1) 상품명 추출
        # -> class 이름으로 상품명 영역 추출
        title_list = info.select('.tit')
        # -> 상품명은 하나만 존재하므로 0번째에 직접 접근하여 텍스트 추출
        item_dict['제품명'] = title_list[0].text.strip()
        
        # 3-2) 가격 추출
        # -> class 이름으로 가격 영역 추출
        price_list = info.select('.num')
        # 가격은 하나만 존재하므로 0번째에 직접 접근하여 텍스트 추출
        price = price_list[0].text.strip()
        # -> 상품가격에 포함된 콤마를 빈 분자열로 변경
        price = price.replace(",","")
        # -> 정수형으로 변환된 결과를 빈 딕셔너리에 추가 
        item_dict['가격'] = int(price)
        
        # 3-3) 스팩 정보 추출
        # a 태그의 경우 한단계 위에서 찾기 
        spec_list = info.select('.detail > a')
        
        # -> 제품별로 스팩항목이 여러개 이므로 추출된 결과만큼 반복
        for v in spec_list :
            v = v.text.strip()         # 텍스트 추출
            tmp = v.split(":")         # 콜론으로 분리 ( ex:디스플레이 : 1920X1080 )
            
            if len(tmp) == 2 :         # 길이가 2라면 key와 value로 분리하여 딕셔너리에 추가 
                key = tmp[0].strip()
                value = tmp[1].strip()
                item_dict[key] = value
        
        # 3-4) 리스트에 추가
        data_list.append(item_dict)
         
            
data_list            


[{'제품명': '파나소닉 루믹스 DC-S1H',
  '가격': 6069990,
  '용도별': '전문가용',
  '유효화소': '2420만화소',
  'ISO': '100~51200',
  '뷰파인더 방식': '전자식',
  '동영상 해상도': '6K',
  '저장매체': 'SD',
  '프레임': '24프레임',
  '특징': '스위블(회전)',
  '스피드': '셔터(1/8000초)',
  '연사': '초당6장',
  '손떨림보정': '바디',
  '화면크기': '3.2인치(8.1cm)',
  '화면화소': '233만화소',
  '디스플레이 형태': '터치',
  '무선기능': '블루투스',
  '스마트': '무선전송(WiFi)',
  '확장': '50~204800',
  '메모리카드': '듀얼슬롯',
  '최대촬영': '약400장(사진)',
  '부피': '1,894cc',
  '이미지센서 부가기능': '로우패스필터제거',
  '무게(배터리제외)': '1052g',
  '무게': '1164g',
  '인터페이스': 'HDMI',
  '부가기능': '방진방적',
  '품목': '미러리스',
  '뷰파인더 형태': '내장뷰파인더'},
 {'제품명': '후지필름 XF10',
  '가격': 482160,
  '최소셔터스피드': '1/4000초',
  '최대조리개': 'F2.8',
  'ISO': 'ISO 51200',
  '최대연속촬영속도': '초당6매',
  '저장매체': 'SD',
  '부가기능': '얼굴인식AF'},
 {'제품명': '후지필름 X-A7',
  '가격': 839990,
  '용도별': '입문자용',
  '유효화소': '2420만화소',
  'ISO': '200~12800',
  '동영상 해상도': 'UHD(4K)',
  '저장매체': 'SD',
  '프레임': '30프레임',
  '특징': '스위블(회전)',
  '스피드': '셔터(1/4000초)',
  '연사': '초당6장',
  '초점방식': '위상차AF',
  '초점영역': '256개

### #05. 수집 결과를 데이터 프레임으로 변환

In [17]:
df = DataFrame(data_list)
df

,제품명,가격,용도별,유효화소,ISO,뷰파인더 방식,동영상 해상도,저장매체,프레임,특징,...,기타,배터리용량,MOS,촬영시간,CCD,줌기능,셔터기능,가이드,전자식뷰파인더종류,바디재질
0,파나소닉 루믹스 DC-S1H,6069990,전문가용,2420만화소,100~51200,전자식,6K,SD,24프레임,스위블(회전),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,후지필름 XF10,482160,NaN,NaN,ISO 51200,NaN,NaN,SD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,후지필름 X-A7,839990,입문자용,2420만화소,200~12800,NaN,UHD(4K),SD,30프레임,스위블(회전),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,후지필름 X-T200,943040,중급자용,2420만화소,200~12800,전자식,UHD(4K),SD,30프레임,틸트+플립(셀카),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,캐논 EOS M50,622420,중급자용,2410만화소,100~25600,전자식,UHD(4K),SD,24프레임,스위블(회전),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,캐논 EOS 7D Mark II,785000,NaN,NaN,100~16000,NaN,Full-HD,CF,60프레임,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,캐논 EOS 1000D,128080,입문자용,NaN,100~1600,NaN,NaN,SD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,소니 미러리스 알파 A5100 16+50mm 렌즈kit,469060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,소니 알파 NEX-6,213800,중급자용,1610만화소,100~25600,전자식,Full-HD,메모리스틱ProDuo,60프레임,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### #06. 엑셀로 저장하기

In [18]:
df.to_excel(keyword + ".xlsx")